# **TEVA Output Explorer Dashboard**

This file mainly manages importing the TEVA output files, running some post-processing functions, setting up plot interactivity, and constructing the dashboard. Two supporting `.py` files contain the post-processing and plotting functions.

`TEVA_Post_Processing.py` contains the post-processing functions that transform the .xlsx file output from TEVA into several more informative and user-friendly data structures. These data structures are used to generate the interactive plots.

`TEVA_Dynamic_Plotting.py` contains functions for plotting the various results of the post-processing functions and handling figure updates when the user interacts with the dashboard controls.

Examples of TEVA output files and observation data are included in the `Sample_Data` folder.

**This notebook is divided into six sections:**
1. Import dependencies
2. Import TEVA output files and observation data
3. Run post-processing functions
4. Dashboard support
5. Dashboard interactivity
6. Dashboard layout

**The dashboard has five main components:**
1. Controls
    - Minimum and maximum sensitivity sliders
    - CC selector for CC feature subplots
    - "Update" button to update tabbed plots
    - "Save" button to export dashboard as a .html file
2. PPV vs. COV plot
3. CC feature subplots
4. Tabbed plots
    - Feature pairing
    - Features used in CCs
    - CCs used in DNFs
5. Confusion matrix for selected CC

<img src='Sample_Data/Example_Dashboard_Layout.png' width='800'>

## Import Dependencies
Import the necessary dependencies and import the `TEVA_Post_Processing.py` and `TEVA_Dynamic_Plotting,py` scripts.

In [ ]:
# Base packages
import numpy as np
import pandas as pd
from matplotlib.pyplot import get_cmap
from matplotlib.colors import rgb2hex

# Holoviews packages
import holoviews as hv
import hvplot.pandas
from bokeh.io import output_notebook
from bokeh.palettes import varying_alpha_palette
from bokeh.plotting import figure
from bokeh.resources import INLINE
from bokeh.models import ColumnDataSource, TabPanel, Tabs
import panel as pn
import colorcet as cc
output_notebook()
hv.extension('bokeh')
pn.extension(throttled=True)

# Custom post processing and plotting functions
import TEVA_Post_Processing as post
import TEVA_Dynamic_Plotting as teva_plot

## Import TEVA Output Files
Import your CC and DNF output files. Selected sheet corresponds to output class. Import the observation data used for running TEVA.

In [ ]:
# # Import CC and DNF output files
ccs = pd.read_excel('Sample_Data/ccs_2DOC_CAMELS_1_S_True_60_60_TEVA007.xlsx', sheet_name='CCEA_High')
dnfs = pd.read_excel('Sample_Data/dnfs_2DOC_CAMELS_1_S_True_60_60_TEVA007.xlsx', sheet_name='DNFEA_High')
# Import observation data
data = pd.read_csv('Sample_Data/test_observations.csv')

## Run Post-Processing Functions
These functions, imported from the `TEVA_Post_Processing.py` file, help parse the Excel spreadsheets into more informative forms for plotting. There are some post-processing functions that are not imported here--they are used in the `TEVA_Dynamic_Plotting.py` file.

In [ ]:
# List of the CCs composing each DNF
all_ccs = post.parse_dnf(dnfs)
all_ccs_flat = post.flatten(all_ccs)

# List of the features composing each CC
cc_features = post.parse_cc(ccs)
unique_features = pd.unique(post.flatten(cc_features))
all_features_flat = post.flatten(cc_features)

# List of the unique CCs across all DNFs
unique_ccs = (np.unique(all_ccs_flat))

# Fitness contours
x_fit, y_fit, z_fit, fitness = post.fitness_contours(1000, dnfs, ccs)

# CC and DNF lengths (needed for a bunch of plotting  related things)
cc_len = np.arange(1, max(ccs['order']) + 1, 1)
dnf_len = np.arange(1, max(dnfs['order']) + 1, 1)

# feature value ranges by cc
feature_values_by_cc = post.feature_ranges_by_cc(ccs)

## Dashboard Support
Select and set up colormaps for the PPV vs. COV plot, fitness contours, etc.
- You can change the colormaps to any Matplotlib named colormap.
- You can change the colormap range depending on the contrast you want
    - Default for CCs and DNFs is 20-220 (full range would be 0-256)

Set up some data structures for plotting. Several other data structures are constructed in the `TEVA_Dynamic_Plotting.py` file.

In [ ]:
# Custom color maps
'''
Create custom colormaps, one for CCs, one from DNFs, and one for fitness contours.
Colormaps can range from 0 to 256, but it is best to trim the lightest and darkest portions out.
'''
cc_colors = []
dnf_colors = []

# CCs and DNFs
for i in range(20,220):
    cc_colors.append(rgb2hex(get_cmap('Blues_r')(i)))
    dnf_colors.append(rgb2hex(get_cmap('Oranges_r')(i)))

# Fitness contours
contour_colors = varying_alpha_palette(color='black', start_alpha=150, end_alpha=10)

# For CC feature heatmap
cc_heatmap_colormap = []
for i in range(0,256):
    cc_heatmap_colormap.append(rgb2hex(get_cmap('Blues')(i)))

# categorical colormap
cat_map = cc.palette['glasbey_bw']

In [ ]:
# Bokeh Data Sources
'''
Bokeh uses a data structure called a "Column Data Source" (CDS) for plotting.
The easiest way to create them with your data is by passing your data as a dictionary.
'''

# column data source for CCs
cc_plot_data = {'x_values': ccs['cov'],
                'y_values': ccs['ppv'],
                'min_sens': ccs['min_feat_sensitivity'],
                'max_sens': ccs['max_feat_sensitivity'],
                'CC': ccs['Unnamed: 0'],
                'Order': ccs['order'],
                'Features': cc_features}
cc_plot_source = ColumnDataSource(data=cc_plot_data)
cc_plot_data= pd.DataFrame(cc_plot_data)

# column data source for DNFs
dnf_plot_data = {'x_values': dnfs['cov'],
                 'y_values': dnfs['ppv'],
                 'Order': dnfs['order'],
                 'DNF': dnfs['Unnamed: 0'],
                 'CCs': all_ccs}
dnf_plot_source = ColumnDataSource(data=dnf_plot_data)

# column data source for fitness contours
dnf_cont_data = {'x_values': x_fit,
                 'y_values': y_fit,
                 'z_values': z_fit}
dnf_cont_source = ColumnDataSource(dnf_cont_data)

## Dashboard Interactivity
Set up the `Widgets` and `Binds` used to drive the dashboard interactivity.

`Widgets` are the controls that the user interacts with (sensitivity sliders, CC selector drop-down list, and the "Update" and "Save" buttons).

`Binds` are functions that bind the value of a `Widget` to a certain function (in this case a dynamic plot). The function for updating the tabbed plots when the user presses the "Update" button comes later, after the app is assembled.

In [ ]:
# WIDGETS
# Dropdown for selecting CC to plot
dropdown_options = list(np.sort(unique_ccs.astype(int)))
cc_select = pn.widgets.Select(options=dropdown_options, width=75, name='CC Select', description='Select a CC to view features.')

# Slider for max sensitivity
sens_slider_max = pn.widgets.IntSlider(name = 'Max. Feature Sensitivity, 10^',
                                       start = int(np.ceil(min(ccs['max_feat_sensitivity']))),
                                       end = 0,
                                       step = 1,
                                       value = 0,
                                       bar_color='#ffffff'
                                       )

# create list of min options
if len(np.floor(ccs['min_feat_sensitivity'][np.isinf(ccs['min_feat_sensitivity'])==False])) == 0:
    slider_options = [-np.inf]
else:
    slider_options = [-np.inf]
    for i in range(int(min(np.floor(ccs['min_feat_sensitivity'][np.isinf(ccs['min_feat_sensitivity'])==False]))), 1):
        slider_options.append(int(i))

sens_slider_min = pn.widgets.DiscreteSlider(name = 'Min. Feature Sensitivity, 10^',
                                            options = slider_options,
                                            value = -np.inf
                                            )

# Tab plot update button
update_button = pn.widgets.Button(name='Update',
                                  button_type='default',
                                  description='Update tabbed plots')

# Button to save to HTML
save_to_html_button = pn.widgets.Button(name='Save',
                                        button_type='success',
                                        description='Save dashboard to HTML file')

def save_to_html(dashboard):
    dashboard.save('TEVA_dashboard.html')


# BINDS
# Bind function to widget
dynamic_subplots = pn.bind(teva_plot.feature_plotter, cc_select, data, cc_features, feature_values_by_cc)
dynamic_confusion_matrix = pn.bind(teva_plot.confusion_matrix_plotter, cc_select, ccs)
dynamic_cc = pn.bind(teva_plot.cc_plotter, sens_slider_min, sens_slider_max, fitness, x_fit, y_fit, z_fit, contour_colors, cc_plot_data, cc_len, cc_plot_source, cc_colors, ccs, dnf_len, dnf_plot_data, dnf_plot_source, dnf_colors, dnfs)

# Initial Tabbed plots
cc_heatmap = teva_plot.cc_heatmap_plotter(cc_heatmap_colormap, unique_features, cc_features, cc_plot_data, sens_slider_min, sens_slider_max)
cc_feature_usage = teva_plot.cc_feature_usage_plot(ccs, cc_plot_data, cc_features, all_features_flat, cat_map, cc_len, sens_slider_min, sens_slider_max)
dnf_usage = teva_plot.dnf_usage_plot(dnfs, dnf_plot_data, cc_plot_data, all_ccs, all_ccs_flat, cat_map, dnf_len, sens_slider_min, sens_slider_max)

tab1 = TabPanel(child=cc_heatmap, title='Feature Pairing')
tab2 = TabPanel(child=cc_feature_usage, title='CC: Feature Usage')
tab3 = TabPanel(child=dnf_usage, title='DNF: CC Usage')

## Dashboard Layout
- Layout the dashboard elements. You can change stylistic elements if you desire.
- Define function to update tabbed plots when the "Update" button is clicked.
- Launch the dashboard in a web browser using local server.

In [ ]:
controls_style = {
    'background': 'cornflowerblue',
    'border': '1px solid black',
    'padding': '10px'
}

app = pn.Column(
    '# TEVA Output Explorer',
    pn.Row('## Controls', sens_slider_min, sens_slider_max, cc_select, update_button, save_to_html_button, height=70, width=1000, styles=controls_style, width_policy='max'),
    pn.Row(dynamic_cc, dynamic_subplots),
    pn.Row(Tabs(tabs=[tab1, tab2, tab3]), dynamic_confusion_matrix))

print(app)

In [ ]:
# Update Tab figures on button click
def update_tab(event):
    '''
    Updates tabbed plots on button click.
    '''
    cc_heatmap = teva_plot.cc_heatmap_plotter(cc_heatmap_colormap, unique_features, cc_features, cc_plot_data, sens_slider_min, sens_slider_max)
    cc_feature_usage = teva_plot.cc_feature_usage_plot(ccs, cc_plot_data, cc_features, all_features_flat, cat_map, cc_len, sens_slider_min, sens_slider_max)
    dnf_usage = teva_plot.dnf_usage_plot(dnfs, dnf_plot_data, cc_plot_data, all_ccs, all_ccs_flat, cat_map, dnf_len, sens_slider_min, sens_slider_max)

    tab1 = TabPanel(child=cc_heatmap, title='Feature Pairing')
    tab2 = TabPanel(child=cc_feature_usage, title='CC: Feature Usage')
    tab3 = TabPanel(child=dnf_usage, title='DNF: CC Usage')
    
    app[3][0] = Tabs(tabs=[tab1, tab2, tab3])

update_button.on_click(update_tab)

update_tab(None)

In [ ]:
# Run dashboard in browser
app.show()